In [1]:
import requests

In [2]:
import time

In [3]:
import csv

In [4]:
manfashion_page_url = "https://tiki.vn/api/personalish/v1/blocks/listings?limit=48&include=advertisement&aggregations=2&trackity_id=848b6aab-1aff-2d7a-ac1b-7ed767d2788e&category=915&page={}&urlKey=thoi-trang-nam"

In [5]:
manfashion_product_url = "https://tiki.vn/api/v2/products/{}"

In [6]:
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"}

In [7]:
product_id_file = "./product_id_file.txt"
product_data_file = "./product_data_file.csv"

In [8]:
def getProductId():
    productIds = []
    i = 1;
    while(True):
        print("Page: ", i)
        response = requests.get(manfashion_page.format(i), headers=headers)
        
        if(response.status_code == 200):
            products = json.loads(response.text)['data']
            if len(products) != 0:
                for product in products:
                    pid = str(product['id'])
                    productIds.append(pid)
            else:
                break
        i = i + 1
        
    return productIds, i    

In [9]:
# productIds, i = getProductId()

In [10]:
def saveProductIds(arrayId = []):
    file = open(product_id_file, "w+")
    str = "\n".join(arrayId)
    file.write(str)
    file.close()

In [11]:
# saveProductIds(productIds)

In [12]:
features = ['discount', 'discount_rate', 'name', 'original_price', 'price', 'rating_average','review_count' ]

In [13]:
def getProductsInfo(arrayId = []):
    products_info = []
    start = 0
    end = 100;
    while start != 6000:
        for id in arrayId[start:end]:
            print(id)
            response = requests.get(manfashion_product_url.format(id), headers=headers)
            if(response.status_code == 200):
                product = response.text
                e = json.loads(product)
                if not e.get("id", False):
                    continue;
                new_e = {}
                for f in features:
                    if(f in e):
                        new_e[f] = e[f]
                    else:
                        new_e[f] = ""
                if('quantity_sold' in e):
                    if(len(e['quantity_sold']) > 0):
                        new_e['quantity_sold'] = e['quantity_sold']['value']
                else:
                    new_e['quantity_sold'] = ''
                products_info.append(new_e)
            time.sleep(3)
        start = end;
        end = end + 100;
    return products_info      

In [14]:
# ps = getProductsInfo(productIds)

In [15]:
# ps[0]

In [16]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
    s = ''
    for c in input_str:
        if c in s1:
            s += s0[s1.index(c)]
        else:
            s += c
    return s

In [17]:
def saveProductsData(products=[]):
    file = open(product_data_file, "w", encoding="utf-8", newline='')
    csv_writer = csv.writer(file)
    count = 0
    for product in products:
        if product is not None:
            if count == 0:
                csv_headers = product.keys()
                csv_writer.writerow(csv_headers)
                count += 1
            p = product
            p['name'] = remove_accents(p['name'])
            csv_writer.writerow(p.values())
    file.close()

In [18]:
# saveProductsData(ps)

In [19]:
productid_csv_file = "./product_id_file.csv"
file = open(product_id_file, "r")
filecsv = open(productid_csv_file, "w", encoding="utf-8", newline='')
csv_writer = csv.writer(filecsv)
csv_writer.writerow(["productid"])
for pid in file.readlines():
    pid = pid.replace("\n", "")
    csv_writer.writerow([pid])
file.close()
filecsv.close()